# [LVV-1615 (v1.0)] - M2 Integration with SAL

This case will verify that the integration of the M2 with SAL.
The blocks below represent the steps of the test case.

Requirements
* EFD
* M2 powered on
* CSC running (either simulation or hardware)

This test will require manual verification of certain events and telemetry in the summit EFD.
Also manual verification of appropriate temperatures for each actuator.

**Make sure you run this notebook on TTS before running at the summit.**

Please, see the [README] file for the requirements to run this notebook.

[LVV-T1802]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T1802
[README]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md

[LVV-1615 (v1.0)]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T1615

## Setting Up Test Environment

Before we run the tests, we want to make sure that we have all the libraries imported, remotes connected, etc.

In [ ]:
test_case = "LVV-T1615"
test_exec = "LVV-EXXXX"

In [ ]:
%load_ext autoreload
%autoreload 2

import asyncio
import os
import yaml

import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy import time 
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from datetime import datetime, timedelta

from lsst_efd_client import EfdClient
from lsst.ts import utils, salobj
from lsst.ts.cRIOpy import M1M3FATable
from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType

import lsst.sitcom.vandv as vandv

from lsst.sitcom import vandv

In [ ]:
exec_info = vandv.ExecutionInfo()
print(exec_info)

print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ["LSST_DDS_DOMAIN_ID"])

Creates a client used to query data from the EFD.

In [ ]:
client = vandv.efd.create_efd_client()

This sets up the logger for the test.

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

In [ ]:
csc_index = 2
hexapod_csc = salobj.Remote(name="MTHexapod", domain=domain, index=csc_index)
print(hexapod_csc)

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

Starts a Script controller which allows putting custom messages into the EFD for later analysis.

In [ ]:
index = 16151296  # Test Case + Test Execution

start_time = datetime.now()
script = salobj.Controller("Script", index=index)

script.log.info(f"START - {test_case} - {test_exec}")

This is how you start the remote for the CSC.

In [ ]:
domain = salobj.Domain()
print(domain)

Create the remote to controle the M2.

In [ ]:
mtm2 = salobj.Remote(name="MTM2", domain=domain)
print(mtm2)

Start the M2 service.

In [ ]:
await mtm2.start_task

Check for heartbeats.

In [ ]:
await mtm2.evt_heartbeat.next(flush=True, timeout=5)

At some point, MTM2 will look for the elevation value.  
Because of this, `mtmount` needs to be in, at least, DISABLED state.

In [ ]:
mtmount = salobj.Remote(name="MTMount", domain=domain)
await asyncio.sleep(10)
await mtmount.start_task

In [ ]:
await salobj.set_summary_state(mtmount, salobj.State.DISABLED)

## M2 DDS Startup Procedure

---
### Connect to https://ls.st/hexrot-vm01

Make sure that you have an IPA account and that your username is part of the saluser group.

Access https://ls.st/hexrot-vm01 using a browser (Firefox recommended).

Log in with your account and open a terminal.

---
### Start MTM2 EUI

Using the terminal, navigate to the `/rubin/mtm2/build folder` and execute `runM2Cntlr`.  
Use the example code below to check if there is another session before starting your own.  

```
$ ps -aux | grep runM2Cntlr
```

If there is another session running, it is recommended that you contact the user and ask them to close it.

---
### MTM2 to DISABLED

Transition the MTM2 CSC into DISABLED state either through LOVE or through Jupyter Notebook. 

In [ ]:
await salobj.set_summary_state(mtm2, salobj.State.DISABLED)

---
Verify the M2 is commandable by DDS by checking the EFD.  
The `MTM2_logevent_commandableByDDS` should publish `True`.

In [ ]:
e = mtcs.rem.mtm2.evt_commandableByDDS.get()
print(e)

Check the event above using the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.logevent_commandableByDDS", 
    fields="state",
    num=1,
)

print(df)

---
Transition the MTM2 CSC into ENABLED state through LOVE. 

In [ ]:
await salobj.set_summary_state(mtm2, salobj.State.ENABLED)

## M2 SAL Telemetry

Verify M2 SAL telemetry against EUI display and the EFD.

If you have been running this notebook sequencially, the items below should be already satisfied.
- Power up the control system
- Transition the CSC to at least disabled 
- Observe the current information in the EUI.

---
In closed loop mode, add 20N to actuator B1.

In [ ]:
axial_forces = np.zeros(72)
axial_forces[0] = 20 # in N - This is Actuator B1 

await mtcs.rem.mtm2.cmd_applyForces.set_start(axial=axial_forces)

Perform data analysis:
- Compare the information from the EFD with the information from the EUI.
- Check all the events & telemetry info available, including, e.g. VMS, temperature, position sensors, etc.
- In the same notebook, we query EFD and compare.

---
Check the `MTM2_position` topic published to the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.position", 
    fields="*",
    num=1,
)

print(df)

---
Check the `MTM2_axialForce` topic published to the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.axialForce", 
    fields="*",
    num=1,
)

print(df)

---
Check the `MTM2_tangentForce` topic published to the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.tangentForce", 
    fields="*",
    num=1,
)

print(df)

---
Check the `MTM2_forceBalance` topic published to the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.forceBalance", 
    fields="*",
    num=1,
)

print(df)

---
Check the `MTM2_netForcesTotal` topic published to the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.netForcesTotal", 
    fields="*",
    num=1,
)

print(df)

---
Check the `MTM2_netMomentsTotal` topic published to the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.netMomentsTotal", 
    fields="*",
    num=1,
)

print(df)

---
Check the `MTM2_temperature` topic published to the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.temperature", 
    fields="*",
    num=1,
)

print(df)

---
Check the `MTM2_zenithAngle` topic published to the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.zenithAngle", 
    fields="*",
    num=1,
)

print(df)

---
Check the `MTM2_axialActutatorSteps` topic published to the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.axialActutatorSteps", 
    fields="*",
    num=1,
)

print(df)

---
Check the `MTM2_tangentActuatorSteps` topic published to the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.tangentActuatorSteps", 
    fields="*",
    num=1,
)

print(df)

---
Check the `MTM2_axialEncoderPositions` topic published to the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.axialEncoderPositions", 
    fields="*",
    num=1,
)

print(df)

---
Check the `MTM2_tangentEncoderPositions` topic published to the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.tangentEncoderPositions", 
    fields="*",
    num=1,
)

print(df)

---
Check the `MTM2_ilcData` topic published to the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.ilcData", 
    fields="*",
    num=1,
)

print(df)

---
Check the `MTM2_powerStatus` topic published to the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.powerStatus", 
    fields="*",
    num=1,
)

print(df)

---
Check the `MTM2_displacementSensors` topic published to the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.displacementSensors", 
    fields="*",
    num=1,
)

print(df)

---
Check the `MTM2_positionIMS` topic published to the EFD.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.positionIMS", 
    fields="*",
    num=1,
)

print(df)

---
Check that the GUI has been updated to match the categorization of the forces in the SAL telemetry.

## M2 SAL Events

### M2 Assembly detailed state

Verify the `MTM2_logevent_detailedState` event is published to the EFD.  
Verify that the event shows all parameters, all values are in the units as defined in the XML and all parameters have meaningful values.  

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.logevent_detailedState", 
    fields="*",
    num=1,
)

print(df)

### M2 Assembly Controller State

Verify the `MTM2_logevent_controllerState` event is published to the EFD.  
Verify that the event shows all parameters, all values are in the units as defined in the XML and all parameters have meaningful values.  

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.logevent_controllerState", 
    fields="*",
    num=1,
)

print(df)

### M2 Assembly inPosition

Verify that `MTM2_logevent_m2AssemblyInPosition` event is published to the EFD.  
Verify that the event shows all parameters, all values are in the units as defined in the XML and all parameters have meaningful values.  

The `MTM2_logevent_m2AssemblyInPosition` event should publish `False`. 

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.logevent_m2AssemblyInPosition", 
    fields="*",
    num=1,
)

print(df)

### Temperature Events

Verify the `MTM2_logevent_CellTemperatureHiWarning` and `MTM2_logevent_temperatureOffset` to the EFD.  
Verify that the event shows all parameters, all values are in the units as defined in the XML and all parameters have meaningful values.

The `MTM2_logevent_CellTemperatureHiWarning` publishes `false` and the `MTM2_logevent_temperatureOffset` is publishing values in the EFD. 

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.logevent_CellTemperatureHiWarning", 
    fields="*",
    num=1,
)

print(df)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.logevent_temperatureOffset", 
    fields="*",
    num=1,
)

print(df)

### Interlock

Verify the `MTM2_logevent_interlock` is published to the EFD.  
Verify that the event shows all parameters, all values are in the units as defined in the XML and all parameters have meaningful values.

The `MTM2_logevent_interlock` event is published. 

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.logevent_interlock", 
    fields="*",
    num=1,
)

print(df)

### TCP/IP connected

<span style="color: firebrick">
    The M2 does not really detect the interlock at this moment.  <br />
    This will be fixed in the C++ version of code.  <br /> 
    <br />
    I guess the TCP/IP fault might not recover (if you mean the cable connects to the cRIO). <br />
    But I might be wrong for this part. <br />
    <br />
</span>
  
Unplug the ethernet cable.  
The system will go into `FAULT` and a `MTM2_logevent_tcpIpConnected` event is published as FALSE.  

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.logevent_tcpIpConnected", 
    fields="*",
    num=1,
)

print(df)

Plug the ethernet cable back in and try to transition from the `FAULT` state to `ENABLED` state.

In [ ]:
await salobj.set_summary_state(mtm2, salobj.State.ENABLED)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.logevent_tcpIpConnected", 
    fields="*",
    num=1,
)

print(df)

### Hardpoint List

Verify the `MTM2_logevent_hardpointList` is published to the EFD.  
Verify that the event shows all parameters, all values are in the units as defined in the XML and  all parameters have meaningful values.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.logevent_hardpointList", 
    fields="*",
    num=1,
)

print(df)

### Inclination Telemetry Source

Verify the `MTM2_logevent_inclinationTelemetrySource` is published in the EFD.  
Verify that the event shows all parameters, all values are in the units as defined in the XML and all parameters have meaningful values.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.logevent_inclinationTelemetrySource", 
    fields="*",
    num=1,
)

print(df)

### Force Balance System Status

Verify the `MTM2_logevent_forceBalanceSystemStatus` is being published to the EFD.  
Verify that the event shows all parameters, all values are in the units as defined in the XML and all parameters have meaningful values.

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.logevent_forceBalanceSystemStatus", 
    fields="*",
    num=1,
)

print(df)

## M2 SAL Commands

### ApplyForces command

Before sending the ApplyForces command, take note of the force telemetry output by the EFD.  
The force telemetry should reflect the forces applied by the LUT.  
This will be seen as the initial condition.  

---
In the enabledState and closed-loop mode, send the `ApplyForces` command over SAL.  
Since this command was applied previously due to a "Test Case Import", we will send another force.  

Forces should not be applied to the axial actuators at the same time as the tangential actuators.  
Make sure the summation of forces in the Z-direction is zero.  
The forces applied should not be random values. As an example, apply bending modes 1-20.

The force telemetry should show the combined value of forces from the initial condition and the forces commanded through SAL.  
The difference between the force telemetry between now and the initial condition should equal the forces sent through the `ApplyForces` command

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.axialForce", 
    fields="*",
    num=1,
)

print(df)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.tangentForce", 
    fields="*",
    num=1,
)

print(df)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.forceBalance", 
    fields="*",
    num=1,
)

print(df)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.netForcesTotal", 
    fields="*",
    num=1,
)

print(df)

In [ ]:
axial_forces = np.zeros(72)
axial_forces[0] = 10 # in N - This is Actuator B1 

await mtcs.rem.mtm2.cmd_applyForces.set_start(axial=axial_forces)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.axialForce", 
    fields="*",
    num=1,
)

print(df)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.tangentForce", 
    fields="*",
    num=1,
)

print(df)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.forceBalance", 
    fields="*",
    num=1,
)

print(df)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.netForcesTotal", 
    fields="*",
    num=1,
)

print(df)

---
### ResetForceOffsets

Send the `ResetForceOffsets` command through SAL.  
This should be done after a successful ApplyForces command.  
Force telemetry should show a difference from when the `ApplyForces` command was issued to after the `ResetForceOffsets` command is issued.  
The 78 nonzero force values are now zero.

In [ ]:
await mtcs.rem.mtm2.cmd_resetForceOffsets.start()

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.axialForce", 
    fields="*",
    num=1,
)

print(df)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.tangentForce", 
    fields="*",
    num=1,
)

print(df)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.forceBalance", 
    fields="*",
    num=1,
)

print(df)

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.netForcesTotal", 
    fields="*",
    num=1,
)

print(df)

### PositionMirror command

<span style="color: firebrick">
    This is a dangerous step.  <br/>
    This actuator can go out of the force range and will be difficult to recover.  <br/>
    This command is an engineering command only.  <br/>
    During observation, only the forces will be used as a measure to move the mirror.  <br/>
    See https://jira.lsstcorp.org/secure/Tests.jspa#/testPlayer/testExecution/LVV-E1008 for the limits.  <br/><br/>
</span>
  
Axis are moved individually.  
  
In the enabled state and closed-loop mode, send a positionMirror command of

```
P1 = (100um,0,0,0,0,0)
P2 = (0,100um,0,0,0,0)
P3 = (0,0,100um,0,0,0)
P4 = (0,0,0,100urad,0,0)
P5 = (0,0,0,0,100urad,0)
P6 = (0,0,0,0,0,100urad)
```

Verify that the event shows all parameters, all values are in the units as defined in the XML and all parameters have meaningful values.  

Note: There will be two types of telemetry - one measured by the hardpoints and one measured by the IMS.
This specific position was selected because it should be within the limits so the `positionMirror` command can be verified without running into an error.  

In [ ]:
await mtcs.rem.mtm2.cmd_positionMirror.set_start(x=100, y=0, z=0, xRot=0, yRot=0, zRot=0)

In [ ]:
await mtcs.rem.mtm2.cmd_positionMirror.set_start(x=0, y=0, z=0, xRot=0, yRot=0, zRot=0)

In [ ]:
await mtcs.rem.mtm2.cmd_positionMirror.set_start(x=0, y=100, z=0, xRot=0, yRot=0, zRot=0)

In [ ]:
await mtcs.rem.mtm2.cmd_positionMirror.set_start(x=0, y=0, z=0, xRot=0, yRot=0, zRot=0)

In [ ]:
await mtcs.rem.mtm2.cmd_positionMirror.set_start(x=0, y=0, z=100, xRot=0, yRot=0, zRot=0)

In [ ]:
await mtcs.rem.mtm2.cmd_positionMirror.set_start(x=0, y=0, z=0, xRot=0, yRot=0, zRot=0)

In [ ]:
await mtcs.rem.mtm2.cmd_positionMirror.set_start(x=0, y=0, z=0, xRot=0.1, yRot=0, zRot=0)

In [ ]:
await mtcs.rem.mtm2.cmd_positionMirror.set_start(x=0, y=0, z=0, xRot=0, yRot=0, zRot=0)

In [ ]:
await mtcs.rem.mtm2.cmd_positionMirror.set_start(x=0, y=0, z=0, xRot=0, yRot=0.1, zRot=0)

In [ ]:
await mtcs.rem.mtm2.cmd_positionMirror.set_start(x=0, y=0, z=0, xRot=0, yRot=0, zRot=0)

In [ ]:
await mtcs.rem.mtm2.cmd_positionMirror.set_start(x=0, y=0, z=0, xRot=0, yRot=0, zRot=0.1)

In [ ]:
await mtcs.rem.mtm2.cmd_positionMirror.set_start(x=0, y=0, z=0, xRot=0, yRot=0, zRot=0)

### ClearError command

<span style="color: firebrick">
    A Mechanical Engineer must be present in order to unplug the actuator.<br/>
</span>

Unplug the cable to actuator A1.  
The M2 CSC goes into a `FAULT` state and is indicated by a red light on the EUI.  
Plug the cable back again.  
  
Send a `clearErrors` command.

In [ ]:
await mtm2.cmd_clearErrors.set_start()

The `clearErrors` command allows the system to transition out of the `FAULT` state and is able to re-enter `Enabled/closed-loop` mode.

### SwtichForceBalanceSystem command

In the `ENABLED` state, send a `switchForceBalanceSystem` command to turn the FB system off.
Verify that the event shows all parameters, all values are in the units as defined in the XML and all parameters have meaningful values.

Note: The Force Balance System is on by default. 

- The `MTM2_command_switchForceBalanceSystem` is accepted and the FB system turns off.
- The `MTM2_logevent_forceBalanceSystemStatus` event publishes false.
- The event shows all parameters, all values are in the units as defined in the XML and all parameters have meaningful values

In [ ]:
await mtm2.cmd_switchForceBalanceSystem.set_start()

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.logevent_forceBalanceSystemStatus", 
    fields="*",
    num=3, # Try to see the changes in events
)

print(df)

In the `ENABLED` state, send a `switchForceBalanceSystem` command to turn the FB system on.

In [ ]:
await mtm2.cmd_switchForceBalanceSystem.set_start()

In [ ]:
df = await client.select_top_n(
    "lsst.sal.MTM2.logevent_forceBalanceSystemStatus", 
    fields="*",
    num=3, # Try to see the changes in events
)

print(df)

### SetTemperatureOffset command

<span style="color: firebrick">
    This will be supported in C++ version. <br />
    <br />
</span>

In the `ENABLED` state, send a `setTemperatureOffsetcommand` with the following parameters:
- ring
- intake
- exhaust
  
Verify that the event shows all parameters, all values are in the units as defined in the XML and all parameters have meaningful values.  
    
The `MTM2_command_setTemperatureOffset` is accepted and the offset of the ring temperatures is changed.  
The `MTM2_logevent_temperatureOffset` event is updated given the new parameters.  
The event shows all parameters, all values are in the units as defined in the XML and all parameters have meaningful values.

In [ ]:
# todo @b1quint: check with Te-Wei for reasonable values
# await mtm2.cmd_setTemperatureOffset.set_start(...)

### selectInclinationSource command

<span style="color: firebrick">
    This will be supported in C++ version. <br />
    <br />
</span>

In the `ENABLED` state, send a `selectInclinationSource` command to choose the MTMount control system as the inclination source.  
Verify that the event shows all parameters, all values are in the units as defined in the XML and all parameters have meaningful values.  

**Note:** The default source is onboard.

The source is changed to the MTMount Control System.  
The `MTM2_logevent_inclinationTelemetrySource` event shows the MTMount Control system as the source.  
The event shows all parameters, all values are in the units as defined in the XML and all parameters have meaningful values.

In [ ]:
# todo @b1quint: check with Te-Wei for reasonable values
# await mtm2.cmd_selectInclinationSource.set_start(...)

## Wrap Up 

Put the relevant components back to STANDBY state.

In [ ]:
await salobj.set_summary_state(mtm2, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(mtmount, salobj.State.STANDBY)